In [1]:
import pickle as pkl
from ruspy.estimation.estimation import estimate

In [4]:
# Visit the documenation for the exact keys necessary or possible.
init_dict_estimation = {
    "model_specifications": {
        "discount_factor": 0.9999,
        "number_states": 90,
        "maint_cost_func": "linear",
        "cost_scale": 1e-3,
    },
    "optimizer": {
        "optimizer_name": "BFGS",
        "use_gradient": "yes",
        "use_search_bounds": "no",
    },
}

In [2]:
# In this demonstration setting, only the data of group 4 is included. For different data sets, either prepare
# yourself from raw data (e.g. https://editorialexpress.com/jrust/research.html) or visit the data repository
# of the OpenSourceCommunity for Rust's (1987) data (https://github.com/OpenSourceEconomics/zurcher-data).
data = pkl.load(open("group_4.pkl", "rb"))

In [6]:
result_transitions, result_fixp = estimate(init_dict_estimation, data)

In [7]:
result_fixp

{'x': array([10.07494188,  2.29309304]),
 'fun': 163.58428365960083,
 'message': 'Optimization terminated successfully.',
 'jac': array([-1.45314713e-06,  3.41241923e-06]),
 '95_conf_interv': array([[12.74172621,  3.35464377],
        [ 7.5056502 ,  1.24275902]]),
 'std_errors': array([1.33422511, 0.55043569])}

In [8]:
result_transitions

{'trans_count': array([1682, 2555,   55]),
 'x': array([0.39189189, 0.59529357, 0.01281454]),
 '95_conf_interv': array([[0.40669652, 0.60997362, 0.01642078],
        [0.37803407, 0.57981038, 0.00994566]]),
 'std_errors': array([0.00737894, 0.0074054 , 0.00167558]),
 'fun': 3140.570557093828}